## Preparing tagged documents

https://datascience.stackexchange.com/questions/25376/better-input-for-doc2vec
https://medium.com/@klintcho/doc2vec-tutorial-using-gensim-ab3ac03d3a1
https://medium.com/@mishra.thedeepak/doc2vec-in-a-simple-way-fa80bfe81104
https://medium.com/@gofortargets/doc2vec-word2vec-in-gensim-c9321c780079

In [1]:
from deathbeds.__github_data_into_dataframes import pulls
from pandas import *; Concat = concat
from CommonMark.render  import renderer
from CommonMark.render.html  import HtmlRenderer
from CommonMark import Parser
from toolz.curried import *
from toolz.curried.operator import *


## Tokenizing markdown messages

In [ ]:
class MarkdownTokenize(renderer.Renderer):
    def __init__(self, *args, **kwargs):
        HtmlRenderer.__init__(self, *args, **kwargs)
        self.codes = []
        
    def code_block(self, node, entering):
        self.codes.append(node.literal)
    
    def __getattr__(self, object):
        def call(node, entering=None):
            self.out(getattr(node, 'literal', None) or '')
        return call
    
    def render(self, object):
        return pandas.Series([super().render(object), self.codes], ['text', 'code'])
    
    softbreak = HtmlRenderer.softbreak

    from gensim.test.utils import common_texts
    from gensim.models.doc2vec import Doc2Vec, TaggedDocument

    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
    model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [36]:
prs = pulls('jupyterlab/jupyterlab', iter=10)

df = Concat(list(prs)).set_index('id')

In [37]:
df = df.body.apply(lambda x: MarkdownTokenize().render(Parser().parse(x))).join(df)

# Extract references to people, issues, and prs.

In [40]:
df = df.text.str.lower().str.split(r'[ ,;.]').apply(
    lambda x: pipe(
        x, filter(compose(contains('@#'), first)),
        groupby(first), valmap(compose(list, map(itemgetter(slice(1, None))))),
        keymap({'@': 'mentions', '#': 'refs'}.__getitem__), pandas.Series)).join(df)

In [42]:
from nltk import tokenize

In [47]:
df = df.text.apply(tokenize.wordpunct_tokenize).apply(
    compose(list, filter(str.isalnum))
).rename('tokens').to_frame().join(df)

In [ ]:
    from gensim.test.utils import common_texts
    from gensim.models.doc2vec import Doc2Vec, TaggedDocument

    documents = [TaggedDocument(doc, [i]) for i, doc in df.tokens.items()]
    model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

C:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
